In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import wandb
import os
from PIL import Image
from torch.utils.data import Dataset

In [ ]:
sweep_config = {
    'method' : 'grid',
    'metric': {
        'name' : 'val_accuracy',
        'goal' : 'maximize'
    },
    'parameters' : {
        'batch_size' : { 'values' : [8, 16]},
        'learning_rate' : { 'values' : [0.001, 0.0001]},
        'hidden_nodes': {'values' : [32, 64]},
        'img_size' : {'values' : [16, 64]},
        'epochs' : {'values': [5, 10]}
    }
}
sweep_id = wandb.sweep(sweep_config, project="5-Flower-Dataset-4")

Create sweep with ID: 7yx61jek
Sweep URL: https://wandb.ai/fangselection123-happiest-minds-technologies/5-Flower-Dataset-3/sweeps/7yx61jek


In [3]:
# =======================
# STEP 0: Initialize wandb
# =======================


# wandb.init(project="alexnet-flowers-v2", config={
#     "epochs": 10,
#     "batch_size": 16,
#     "learning_rate": 0.001,
#     "architecture": "alexnet",
#     "pretrained": True,
#     "input_size": 128
# })

# Shortcut to config values
# config = wandb.config

In [4]:


class CustomFlowersDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_to_idx = {}

        class_names = sorted(os.listdir(root_dir))
        self.class_to_idx = {class_name: idx for idx, class_name in enumerate(class_names)}
        for class_name in class_names:
            class_dir = os.path. join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                img_path =os.path.join(class_dir, img_name)
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.image_paths.append(img_path)
                    self.labels.append(self.class_to_idx[class_name])
        
    def __len__(self):
        return len(self.image_paths)
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


In [5]:
# import os

# path = "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/Data/flowers/train/.DS_Store"
# path = "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/Data/flowers/val/.DS_Store"

# # Remove .DS_Store if it exists
# if os.path.exists(path):
#     os.remove(path)
#     print(f"Deleted: {path}")
# else:
#     print(f"File does not exist: {path}")


In [ ]:
def train():
    with wandb.init() as run:
        config= wandb.config
        train_dir="Data/flowers/train"
        test_dir="Data/flowers/val"
        transform = transforms.Compose([
            transforms.Resize((config.img_size, config.img_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])



        train_dataset = CustomFlowersDataset(train_dir, transform=transform)
        test_dataset = CustomFlowersDataset(test_dir, transform=transform)
        train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)


        class SimpleModel(nn.Module):
            def __init__(self):
                super(SimpleModel, self).__init__()
                self.flatten = nn.Flatten()
                self.fc1 = nn.Linear(config.img_size * config.img_size * 3, config.hidden_nodes)
                self.bn1 = nn.BatchNorm1d(config.hidden_nodes)
                # Output layer for 5 classes
                self.relu = nn.ReLU()
                self.dropout = nn.Dropout(p=0.2)
                self.fc2 = nn.Linear(config.hidden_nodes, 5)

            def forward(self, x):
                x = self.flatten(x)
                x = self.fc1(x)
                x= self.bn1(x)
                x = self.relu(x)
                x = self.dropout(x)
                x = self.fc2(x)
                return x  # No softmax needed; use CrossEntropyLoss
        model = SimpleModel()
        # -----------------------------
        # Training Setup
        # -----------------------------
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=1e-5) # l2 regularization
        # shedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

        # -----------------------------
        # Training Loop
        # -----------------------------
        EPOCHS = 10
        for epoch in range(EPOCHS):
            model.train()
            train_loss = 0
            train_correct = 0
            train_total = 0

            for i, (images, labels) in enumerate(train_loader):
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)
                # L1 regularization (manual)
                # l1_lambda = 1e-5
                # l1_loss = sum(torch.sum(torch.abs(param)) for param in model.parameters())
                # loss += l1_lambda * l1_loss

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                # shedule.step()

                train_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                batch_correct = (preds == labels).sum().item()
                train_correct += batch_correct
                train_total += labels.size(0)
                # Print every 10 batches
                if(i + 1) % 10 == 0:
                    batch_acc = batch_correct / labels.size(0)
                    print(f"[Batch {i+1}/{len(train_loader)}] Loss: {loss.item():.4f}, Batch Acc: {batch_acc:.4f}")

            train_accuracy = train_correct / train_total
            wandb.log({"epoch": epoch + 1, "train_loss": train_loss, "train_accuracy": train_accuracy})
            print(f"Epoch {epoch+1} Summary - Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

            # -----------------------------
            # # Evaluation (Optional)
            # # -----------------------------
        model.eval()
        test_correct = 0
        test_total = 0
        test_loss = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                test_correct += (preds == labels).sum().item()
                test_total += labels.size(0)
        test_accuracy = test_correct / test_total
        wandb.log({"test_loss": test_loss, "test_accuracy": test_accuracy})
        print(f"Test Accuracy: {test_correct / test_total:.4f}")


        




In [ ]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: 12vwi9s4 with config:
wandb: 	batch_size: 8
wandb: 	epochs: 5
wandb: 	hidden_nodes: 32
wandb: 	img_size: 16
wandb: 	learning_rate: 0.001
wandb: Currently logged in as: fangselection123 (fangselection123-happiest-minds-technologies) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[Batch 10/501] Loss: 1.7272, Batch Acc: 0.2500
[Batch 20/501] Loss: 1.4052, Batch Acc: 0.2500
[Batch 30/501] Loss: 1.7162, Batch Acc: 0.2500
[Batch 40/501] Loss: 1.5729, Batch Acc: 0.2500
[Batch 50/501] Loss: 1.4287, Batch Acc: 0.2500
[Batch 60/501] Loss: 1.8150, Batch Acc: 0.1250
[Batch 70/501] Loss: 1.7172, Batch Acc: 0.2500
[Batch 80/501] Loss: 1.5038, Batch Acc: 0.2500
[Batch 90/501] Loss: 1.5879, Batch Acc: 0.3750
[Batch 100/501] Loss: 1.5946, Batch Acc: 0.0000
[Batch 110/501] Loss: 1.5876, Batch Acc: 0.1250
[Batch 120/501] Loss: 1.4080, Batch Acc: 0.3750
[Batch 130/501] Loss: 1.6717, Batch Acc: 0.2500
[Batch 140/501] Loss: 1.6309, Batch Acc: 0.2500
[Batch 150/501] Loss: 1.5443, Batch Acc: 0.5000
[Batch 160/501] Loss: 1.6434, Batch Acc: 0.2500
[Batch 170/501] Loss: 1.5402, Batch Acc: 0.3750
[Batch 180/501] Loss: 1.6099, Batch Acc: 0.3750
[Batch 190/501] Loss: 1.6427, Batch Acc: 0.1250
[Batch 200/501] Loss: 1.6367, Batch Acc: 0.1250
[Batch 210/501] Loss: 1.5850, Batch Acc: 0.6250
[

epoch,▁▂▃▃▄▅▆▆▇█
test_accuracy,▁
test_loss,▁
train_accuracy,▆▆▆▅▄▂▁▄▇█
train_loss,█▅▃▁▇▅▇▆▁▆
epoch,10
test_accuracy,0.2908
test_loss,202.09772
train_accuracy,0.2747
train_loss,805.1322


wandb: Agent Starting Run: wqb11ybm with config:
wandb: 	batch_size: 8
wandb: 	epochs: 5
wandb: 	hidden_nodes: 32
wandb: 	img_size: 16
wandb: 	learning_rate: 0.0001


[Batch 10/501] Loss: 1.5158, Batch Acc: 0.3750
[Batch 20/501] Loss: 1.9206, Batch Acc: 0.2500
[Batch 30/501] Loss: 1.5109, Batch Acc: 0.5000
[Batch 40/501] Loss: 1.7560, Batch Acc: 0.2500
[Batch 50/501] Loss: 1.6506, Batch Acc: 0.2500
[Batch 60/501] Loss: 1.6553, Batch Acc: 0.2500
[Batch 70/501] Loss: 1.5192, Batch Acc: 0.3750
[Batch 80/501] Loss: 1.9761, Batch Acc: 0.1250
[Batch 90/501] Loss: 1.6270, Batch Acc: 0.3750
[Batch 100/501] Loss: 1.9550, Batch Acc: 0.0000
[Batch 110/501] Loss: 1.7478, Batch Acc: 0.1250
[Batch 120/501] Loss: 1.7697, Batch Acc: 0.2500
[Batch 130/501] Loss: 1.7731, Batch Acc: 0.2500
[Batch 140/501] Loss: 1.5832, Batch Acc: 0.2500
[Batch 150/501] Loss: 1.9784, Batch Acc: 0.0000
[Batch 160/501] Loss: 1.7573, Batch Acc: 0.1250
[Batch 170/501] Loss: 1.6795, Batch Acc: 0.2500
[Batch 180/501] Loss: 1.7029, Batch Acc: 0.3750
[Batch 190/501] Loss: 1.3742, Batch Acc: 0.2500
[Batch 200/501] Loss: 1.7429, Batch Acc: 0.2500
[Batch 210/501] Loss: 1.7847, Batch Acc: 0.0000
[

...

wandb: Agent Starting Run: grmgdkal with config:
wandb: 	batch_size: 8
wandb: 	epochs: 10
wandb: 	hidden_nodes: 64
wandb: 	img_size: 64
wandb: 	learning_rate: 0.0001


[Batch 10/501] Loss: 1.6625, Batch Acc: 0.1250
[Batch 20/501] Loss: 1.9010, Batch Acc: 0.1250
[Batch 30/501] Loss: 1.7465, Batch Acc: 0.1250
[Batch 40/501] Loss: 1.7834, Batch Acc: 0.1250
[Batch 50/501] Loss: 1.4060, Batch Acc: 0.5000
[Batch 60/501] Loss: 1.6924, Batch Acc: 0.2500
[Batch 70/501] Loss: 1.6963, Batch Acc: 0.3750
[Batch 80/501] Loss: 1.3248, Batch Acc: 0.3750
[Batch 90/501] Loss: 1.8852, Batch Acc: 0.2500
[Batch 100/501] Loss: 1.7041, Batch Acc: 0.2500
[Batch 110/501] Loss: 1.5929, Batch Acc: 0.0000
[Batch 120/501] Loss: 1.5271, Batch Acc: 0.2500
[Batch 130/501] Loss: 1.8167, Batch Acc: 0.3750
[Batch 140/501] Loss: 1.4179, Batch Acc: 0.5000
[Batch 150/501] Loss: 1.7196, Batch Acc: 0.1250
[Batch 160/501] Loss: 1.7613, Batch Acc: 0.3750
[Batch 170/501] Loss: 1.6134, Batch Acc: 0.1250
[Batch 180/501] Loss: 1.7449, Batch Acc: 0.1250
[Batch 190/501] Loss: 1.5810, Batch Acc: 0.3750
[Batch 200/501] Loss: 1.7712, Batch Acc: 0.2500
[Batch 210/501] Loss: 1.5792, Batch Acc: 0.1250
[

epoch,▁▂▃▃▄▅▆▆▇█
test_accuracy,▁
test_loss,▁
train_accuracy,▃█▃▁▂▂▁▃▆█
train_loss,▆▂█▄▅▅▅▄▃▁
epoch,10
test_accuracy,0.30564
test_loss,201.63809
train_accuracy,0.29591
train_loss,817.27861


wandb: Agent Starting Run: ishonvie with config:
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	hidden_nodes: 32
wandb: 	img_size: 16
wandb: 	learning_rate: 0.001


[Batch 10/251] Loss: 1.6618, Batch Acc: 0.2500
[Batch 20/251] Loss: 1.7765, Batch Acc: 0.2500
[Batch 30/251] Loss: 1.7056, Batch Acc: 0.1250
[Batch 40/251] Loss: 1.6525, Batch Acc: 0.2500
[Batch 50/251] Loss: 1.7577, Batch Acc: 0.2500
[Batch 60/251] Loss: 1.5491, Batch Acc: 0.3750
[Batch 70/251] Loss: 1.8893, Batch Acc: 0.2500
[Batch 80/251] Loss: 1.7818, Batch Acc: 0.2500
[Batch 90/251] Loss: 1.6115, Batch Acc: 0.2500
[Batch 100/251] Loss: 1.5596, Batch Acc: 0.2500
[Batch 110/251] Loss: 1.6213, Batch Acc: 0.1250
[Batch 120/251] Loss: 1.8042, Batch Acc: 0.1875
[Batch 130/251] Loss: 1.6564, Batch Acc: 0.1875
[Batch 140/251] Loss: 1.6303, Batch Acc: 0.3750
[Batch 150/251] Loss: 1.8399, Batch Acc: 0.0625
[Batch 160/251] Loss: 1.8478, Batch Acc: 0.1250
[Batch 170/251] Loss: 1.5560, Batch Acc: 0.2500
[Batch 180/251] Loss: 1.3204, Batch Acc: 0.4375
[Batch 190/251] Loss: 1.4309, Batch Acc: 0.4375
[Batch 200/251] Loss: 1.6626, Batch Acc: 0.3125
[Batch 210/251] Loss: 1.6200, Batch Acc: 0.3125
[

In [ ]:
# print(len(train_dataset), len(test_dataset))
# print(len(train_loader), len(test_loader))
# for img, label in train_dataset:
#     print(img.shape, label)
#     break
# for img, label in test_dataset:
#     print(img.shape, label)
#     break
# for img, label in train_loader:
#     print(img.shape, label)
#     break   

# for img, label in test_loader:
#     print(img.shape, label)
#     break


4008 1011
251 64
torch.Size([3, 128, 128]) 0
torch.Size([3, 128, 128]) 0
torch.Size([16, 3, 128, 128]) tensor([2, 3, 4, 1, 2, 1, 1, 0, 0, 3, 1, 1, 4, 3, 0, 2])
torch.Size([16, 3, 128, 128]) tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [ ]:
wandb.agent(sweep_id, function=train)

In [ ]:
# -----------------------------
# Training Loop
# -----------------------------
EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    train_correct = 0
    train_total = 0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        train_correct += (preds == labels).sum().item()
        train_total += labels.size(0)
        # Print every 10 batches
        if(i + 1) % 10 == 0:
            batch_acc = train_correct / labels.size(0)
            print(f"[Batch {i+1}/{len(train_loader)}] Loss: {loss.item():.4f}, Batch Acc: {batch_acc:.4f}")


    train_accuracy = train_correct / train_total
    wandb.log({"epoch": epoch + 1, "train_loss": train_loss, "train_accuracy": train_accuracy})
    print(f"Epoch {epoch+1} Summary - Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

# -----------------------------
# # Evaluation (Optional)
# # -----------------------------
model.eval()
test_correct = 0
test_total = 0
test_loss = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        test_correct += (preds == labels).sum().item()
        test_total += labels.size(0)

print(f"Test Accuracy: {test_correct / test_total:.4f}")



[Batch 10/251] Loss: 24.2275, Batch Acc: 3.4375
[Batch 20/251] Loss: 11.9568, Batch Acc: 6.3750
[Batch 30/251] Loss: 22.4465, Batch Acc: 9.9375
[Batch 40/251] Loss: 9.7260, Batch Acc: 13.3750
[Batch 50/251] Loss: 11.5228, Batch Acc: 17.3750
[Batch 60/251] Loss: 11.9649, Batch Acc: 21.7500
[Batch 70/251] Loss: 6.5767, Batch Acc: 25.5625
[Batch 80/251] Loss: 17.2534, Batch Acc: 29.6250
[Batch 90/251] Loss: 16.7764, Batch Acc: 34.6250
[Batch 100/251] Loss: 15.6186, Batch Acc: 38.3750
[Batch 110/251] Loss: 18.6942, Batch Acc: 42.0000
[Batch 120/251] Loss: 7.5438, Batch Acc: 46.1250
[Batch 130/251] Loss: 9.0678, Batch Acc: 50.6875
[Batch 140/251] Loss: 20.6536, Batch Acc: 54.2500
[Batch 150/251] Loss: 16.8641, Batch Acc: 58.2500
[Batch 160/251] Loss: 20.7025, Batch Acc: 62.2500
[Batch 170/251] Loss: 10.8953, Batch Acc: 66.8750
[Batch 180/251] Loss: 13.8928, Batch Acc: 70.9375
[Batch 190/251] Loss: 20.1441, Batch Acc: 73.8750
[Batch 200/251] Loss: 10.8839, Batch Acc: 77.8125
[Batch 210/251] 